In [1]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/catalog-subset/attr_subset.csv


In [2]:
import pandas as pd
import numpy as np
import warnings


import matplotlib.pyplot as plt
import io
import keras

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',100)
pd.set_option('display.max_colwidth',200)

!pip install tf_keras

!wget --quiet  https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/tools/tokenization.py   
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import warnings

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub 
import tokenization

from sklearn.metrics import accuracy_score, roc_curve, auc, classification_report, roc_auc_score, precision_recall_curve,confusion_matrix
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


  Obtaining dependency information for tf_keras from https://files.pythonhosted.org/packages/19/26/ca8a6cca61f2a44f1e7ee71ebdb9c8dfbc4371f418db811cdca4641f6daa/tf_keras-2.15.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.8 MB/s eta 0:00:00


In [3]:
data = pd.read_csv('/kaggle/input/catalog-subset/attr_subset.csv')
print(data.shape)
data.head()

(506287, 8)


,asin,item_name,prep_type,is_set,certified_level,pack_ind,number_items_in_set,set_target
0,B01IBY1PI2,"Vertx Mens Tactical Pants Cargo Utility with Pockets, Lightweight Casual Outdoor Work-Wear, Relaxed-Fit, Phantom LT 2.0, Smoke Grey, 31x30",bagging,Y,1.0,0,1.0,0
1,B07TCTNF96,"Reebok Workout Ready Knit Pant, Black, XX-Large",bagging,N,1.0,0,NaN,0
2,B07Z86K31S,"Doctor's Choice Compression Low Calf Crew Socks, Plantar Fasciitis, Achilles Tendonitis & Arch Support for Men & Women, 1 Pair, Tan/Large",bagging,N,1.0,0,NaN,0
3,B08SBGY2TC,"Softball Drawstring Bag for Girls, Eat Sleep Softball Repeat Backpack, Softball Player Gift, Teen Sport Pack Cinch Sack Tote Bag",bagging,N,1.0,1,NaN,0
4,B08DP4R7WX,"BioSteel Sports Drink, Great Tasting Hydration with 5 Essential Electrolytes, Blue Raspberry Flavor, 16.7 Fluid Ounces, 12-Pack",asinstk,N,1.0,1,12.0,0


In [4]:
data['item_name'] = data['item_name'].fillna('unknown')
data.isnull().sum()

asin                        0
item_name                   0
prep_type                   0
is_set                      0
certified_level             1
pack_ind                    0
number_items_in_set    262700
set_target                  0
dtype: int64

In [5]:
def bert_encode(texts, tokenizer, max_len = 128):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
        
        text = text[:max_len-2]
        input_sequence = ['[CLS]'] + text + ['[SEP]']
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
        
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [6]:
def build_model(bert_layer, max_len = 128):
    
    input_word_ids = Input(shape = (max_len,), dtype = tf.int32, name = 'input_word_ids')
    input_mask = Input(shape = (max_len, ), dtype = tf.int32, name = 'input_mask')
    segments_ids = Input(shape = (max_len,), dtype = tf.int32, name = 'segment_ids')
    
    _, sequence_output = bert_layer([input_word_ids, input_mask, segments_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation = 'sigmoid')(clf_output)
    
    model = Model(inputs = [input_word_ids, input_mask, segments_ids], outputs = out)
    model.compile(Adam(lr = 2e-6), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

# Load and pre-processing

In [7]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 21.1 s, sys: 4.72 s, total: 25.9 s
Wall time: 28.2 s


In [8]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

# Train/test split

In [9]:
data.set_target.value_counts()

set_target
0    487817
1     18470
Name: count, dtype: int64

# Downsample

In [10]:
set_dt = data[data["set_target"] == 1]
nonset_dt  = data[data["set_target"] == 0]
print(set_dt.shape)
print(nonset_dt.shape)

(18470, 8)
(487817, 8)


In [11]:
from sklearn.utils import resample
nonset_ds = resample(nonset_dt,
             replace=True,
             n_samples=len(set_dt),
             random_state=42)

print(nonset_ds.shape)

(18470, 8)


In [12]:
data_sub = pd.concat([set_dt, nonset_ds])
print(data_sub.shape)
data_sub.set_target.value_counts()

(36940, 8)


set_target
1    18470
0    18470
Name: count, dtype: int64

In [13]:
data_smp = data_sub.sample(5000, random_state = 42)
data_smp.set_target.value_counts()

set_target
0    2500
1    2500
Name: count, dtype: int64

In [14]:
X = data_smp.item_name
y = data_smp.set_target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2023)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:',y_train.shape)
print('y_test shape:',y_test.shape)

X_train shape: (4000,)
X_test shape: (1000,)
y_train shape: (4000,)
y_test shape: (1000,)


In [15]:
%%time

train_input = bert_encode(X_train.values, tokenizer, max_len = 128)
test_input = bert_encode(X_test.values, tokenizer, max_len = 128)
train_labels = y_train.values

CPU times: user 2.89 s, sys: 2.36 ms, total: 2.89 s
Wall time: 2.89 s


# Modeling

In [16]:
model = build_model(bert_layer, max_len = 128)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_word_ids (InputLayer  [(None, 128)]                0         []                            
 )                                                                                                
                                                                                                  
 input_mask (InputLayer)     [(None, 128)]                0         []                            
                                                                                                  
 segment_ids (InputLayer)    [(None, 128)]                0         []                            
                                                                                                  
 keras_layer (KerasLayer)    [(None, 1024),               3351418   ['input_word_ids[0][0]',  

In [17]:
%%time

train_history = model.fit(train_input, train_labels, validation_split = 0.2, epochs = 3, batch_size = 16)
model.save('model.h5')

Epoch 1/3
200/200 [==============================] - 351s 1s/step - loss: 0.7944 - accuracy: 0.4850 - val_loss: 0.7103 - val_accuracy: 0.4850
Epoch 2/3
200/200 [==============================] - 262s 1s/step - loss: 0.7132 - accuracy: 0.4934 - val_loss: 0.7070 - val_accuracy: 0.4850
Epoch 3/3
200/200 [==============================] - 262s 1s/step - loss: 0.7056 - accuracy: 0.5047 - val_loss: 0.6930 - val_accuracy: 0.5150
CPU times: user 10min 30s, sys: 4min 14s, total: 14min 44s
Wall time: 14min 42s


In [18]:
%%time
train_pred = model.predict(train_input)

125/125 [==============================] - 111s 869ms/step
CPU times: user 1min 37s, sys: 14.1 s, total: 1min 51s
Wall time: 1min 51s


In [19]:
auc_train = roc_auc_score(y_train, train_pred)
print('train roc:', auc_train )

train roc: 0.49428998715247113


In [20]:
print('accuracy score: \n', accuracy_score(y_train, np.round(train_pred)), '\n')

print('classification report \n', classification_report(y_train, np.round(train_pred)), '\n')

print('confusion matrix \n', confusion_matrix(y_train, np.round(train_pred)), '\n')

accuracy score: 
 0.49925 

classification report 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00      2003
           1       0.50      1.00      0.67      1997

    accuracy                           0.50      4000
   macro avg       0.25      0.50      0.33      4000
weighted avg       0.25      0.50      0.33      4000
 

confusion matrix 
 [[   0 2003]
 [   0 1997]] 



In [21]:
%%time
test_pred = model.predict(test_input)

32/32 [==============================] - 29s 865ms/step
CPU times: user 26.2 s, sys: 3.45 s, total: 29.6 s
Wall time: 29.6 s


In [22]:
test_pred

array([[0.502928  ],
       [0.502928  ],
       [0.50292796],
       [0.502928  ],
       [0.502928  ],
       [0.502928  ],
       [0.50292796],
       [0.502928  ],
       [0.502928  ],
       [0.502928  ],
       [0.502928  ],
       [0.502928  ],
       [0.502928  ],
       [0.50292796],
       [0.502928  ],
       [0.502928  ],
       [0.5029281 ],
       [0.5029281 ],
       [0.502928  ],
       [0.5029281 ],
       [0.502928  ],
       [0.5029281 ],
       [0.502928  ],
       [0.502928  ],
       [0.502928  ],
       [0.502928  ],
       [0.502928  ],
       [0.5029281 ],
       [0.5029281 ],
       [0.502928  ],
       [0.50292796],
       [0.5029281 ],
       [0.502928  ],
       [0.502928  ],
       [0.50292796],
       [0.5029281 ],
       [0.502928  ],
       [0.502928  ],
       [0.502928  ],
       [0.50292796],
       [0.50292796],
       [0.502928  ],
       [0.502928  ],
       [0.5029281 ],
       [0.5029281 ],
       [0.502928  ],
       [0.502928  ],
       [0.502

In [23]:
auc = roc_auc_score(y_test, test_pred)
print('test roc:', auc)

test roc: 0.4736610517978648


In [24]:
print('accuracy score: \n', accuracy_score(y_test, np.round(test_pred)), '\n')

print('classification report \n', classification_report(y_test, np.round(test_pred)), '\n')

print('confusion matrix \n', confusion_matrix(y_test, np.round(test_pred)), '\n')

accuracy score: 
 0.503 

classification report 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       497
           1       0.50      1.00      0.67       503

    accuracy                           0.50      1000
   macro avg       0.25      0.50      0.33      1000
weighted avg       0.25      0.50      0.34      1000
 

confusion matrix 
 [[  0 497]
 [  0 503]] 

